In [31]:
import time
import math

# Utils

In [32]:
# Read data base
def loadDatabase(path):
    database = open(path, 'r')
    data = list()
    line = database.readline()
    while line != "":
        dataSet = list(map(int, line.split()))
        data.append(dataSet)
        line = database.readline()
    return data

In [33]:
# Check if itemset is in a record
def findInRecord(itemset, record):
    for i in itemset:
        if i not in record:
            return False
    return True

In [34]:
# Count the number of occurence of itemset in data
def count(itemset, data):
    counter = 0
    for record in data:
        if findInRecord(itemset, record):
            counter += 1
    return counter

# Apriori algorithm


```
Let k=1
Generate frequent itemsets of length 1
Repeat until no new frequent itemsets are identified
    step1: Generate length (k+1) candidate itemsets from length k frequent itemsets
    step2: Prune candidate itemsets containing subsets of length k that are infrequent
    step3: Count the support of each candidate by scanning the DB
            and eliminate candidates that are infrequent, leaving only those that are frequent
```

In [35]:
# merge two itemset (if possible)
def merge(itemset1, itemset2):
    newFreqItem = set(itemset1).union(set(itemset2))
    if len(newFreqItem) == (len(itemset1)+1):
        return newFreqItem


# generate candidates of length (k+1) by merging two frequent itemsets of length k
def generateCandidates(oldCandidates):
    newCandidates = set()
    for i in range(len(oldCandidates)):
        for j in range(i + 1, len(oldCandidates)):
            newItemset = merge(oldCandidates[i], oldCandidates[j])
            # if merge is success, add it to newCandidate set
            if newItemset is not None:
                newCandidates.add(tuple(sorted(tuple(newItemset))))
    # conver the set to a list
    result = list()
    for i in newCandidates:
        result.append(sorted(i))
    return sorted(result)


# check whether a itemset has infrequent subsets
def isFrequentSubset(itemset, old_candidates):
    for i in range(len(itemset)):
        subset = list(itemset)
        del subset[i]
        if subset not in old_candidates:
            return False
    return True


# prune candidates which has infrequent subsets
def prune(candidates, oldCandidates):
    afterPrune = list()
    for itemset in candidates:
        if isFrequentSubset(itemset, oldCandidates):
            afterPrune.append(itemset)
    return afterPrune


# generate the frequent items of length 1
def length_1_freqItemSet(data, minsup):
    items = dict()
    for record in data:
        for i in record:
            if i in items:
                items[i] += 1
            else:
                items[i] = 1

    freq_items = list()
    for item in items:
        if items[item] >= minsup:
            freq_items.append([[item], items[item]])
    return sorted(freq_items)


# data is a list of transaction, minsup is the minimum support
def aprioriAlgorithm(data, minsup):
    #  generate frequent itemsets of size 1
    len_1_frq = length_1_freqItemSet(data, minsup)

    freq_itemsets = list(len_1_frq)
    candidates = []
    for item in len_1_frq:
        candidates.append(item[0])

    while len(candidates) > 0:
        # length K+1 candidates
        nextCandidates = generateCandidates(candidates)
        # remove infrequent candidates
        nextCandidates = prune(nextCandidates, candidates)
        # count and kick away infrequent candidate
        candidates = list()
        for itemSet in nextCandidates:
            if count(itemSet, data) >= minsup:
                candidates.append(itemSet)
                freq_itemsets.append([itemSet, count(itemSet, data)])
    return freq_itemsets

### Run

In [36]:
# please refer to apriori.py for the implementation
data = loadDatabase('a2dataset.txt')
print("Length data: ", len(data))
minsup = 100

start = time.time() 
freq_itemsets = aprioriAlgorithm(data, minsup)
print("Frequent itemsets: ", freq_itemsets)
print("Length frequent itemsets: ", len(freq_itemsets))
print("Time: ", time.time() - start)

Length data:  2000
Frequent itemsets:  [[[32], 290], [[38], 436], [[39], 1153], [[41], 485], [[48], 870], [[32, 39], 167], [[32, 48], 150], [[38, 39], 279], [[38, 41], 165], [[38, 48], 194], [[39, 41], 367], [[39, 48], 607], [[41, 48], 258], [[38, 39, 41], 129], [[38, 39, 48], 144], [[39, 41, 48], 204]]
Length frequent itemsets:  16
Time:  0.01598358154296875


# Find closed and max itemsets with apriori algorithm


In [37]:
# check if largeSet is immediate superset of smallSet
def isImmediateSuperSet(smallSet, largeSet):
    if largeSet.issuperset(smallSet):
        if len(largeSet) - len(smallSet) == 1:
            return True
    return False


data = loadDatabase('a2dataset.txt')
minsup = 100

start = time.time()
# get frequent Itemset from external library
dataWithFreq = aprioriAlgorithm(data, minsup)

freqItemSet = []
for item in dataWithFreq:
    freqItemSet.append(item[0])

dataWithFreq = {tuple(dataWithFreq[i][0]): dataWithFreq[i][1] for i in range(len(dataWithFreq))}

# find closed and max itemset
closedItemSet = list()
maxItemSet = list()


# go though every freqitem set, and check if it is closed of max
for x, y in dataWithFreq.items():
    targetSet = set(x)
    freq = y
    isClosedFreqItemSet = True
    isMaxSet = True
    # compare a itemSet to all other itemSet
    for p, q in dataWithFreq.items():
        comparingSet = set(p)
        comparingfreq = q
        # do not do self comparison
        if targetSet == comparingSet:
            continue
        if isImmediateSuperSet(targetSet, comparingSet):
            isMaxSet = False
            if comparingfreq == freq:
                isClosedFreqItemSet = False
                break
    if isClosedFreqItemSet:
        closedItemSet.append(list(targetSet))
    if isMaxSet:
        maxItemSet.append(list(targetSet))

        
print("ClosedItemSets are: \n")
print(closedItemSet)
print("Length closed itemsets: ", len(closedItemSet))

print("\nMaxItemSets are:")
print(maxItemSet)
print("Length max itemsets: ", len(maxItemSet))

print("Time: ", time.time() - start)

ClosedItemSets are: 

[[32], [38], [39], [41], [48], [32, 39], [32, 48], [38, 39], [41, 38], [48, 38], [41, 39], [48, 39], [48, 41], [41, 38, 39], [48, 38, 39], [48, 41, 39]]
Length closed itemsets:  16

MaxItemSets are:
[[32, 39], [32, 48], [41, 38, 39], [48, 38, 39], [48, 41, 39]]
Length max itemsets:  5
Time:  0.014995336532592773


# Apriori algorithm is not very efficient. 

In [38]:
# check if largeSet is immediate superset of smallSet
def isImmediateSuperSet(smallSet, largeSet):
    if largeSet.issuperset(smallSet):
        if len(largeSet) - len(smallSet) == 1:
            return True
    return False

data = loadDatabase('a2dataset.txt')
minsup = 100
theta = 0.2

start = time.time()
# get frequent Itemset from external library
dataWithFreq = aprioriAlgorithm(data, minsup)

freqItemSet = []
for item in dataWithFreq:
    freqItemSet.append(item[0])

dataWithFreq = {tuple(dataWithFreq[i][0]): dataWithFreq[i][1] for i in range(len(dataWithFreq))}

# find closed and max itemset
closedItemSet = list()
maxItemSet = list()


# go though every freqitem set, and check if it is closed of max
for x, y in dataWithFreq.items():
    targetSet = set(x)
    freq = y
    isClosedFreqItemSet = True
    isMaxSet = True
    # compare a itemSet to all other itemSet
    for p, q in dataWithFreq.items():
        comparingSet = set(p)
        comparingfreq = q
        # do not do self comparison
        if targetSet == comparingSet:
            continue
        if isImmediateSuperSet(targetSet, comparingSet):
            isMaxSet = False
            # Compare distance patern with theta
            if  theta <= 1 - comparingfreq / freq:
                isClosedFreqItemSet = False
                break
    if isClosedFreqItemSet:
        closedItemSet.append(list(targetSet))
    if isMaxSet:
        maxItemSet.append(list(targetSet))

        
print("ClosedItemSets are: \n")
print(closedItemSet)
print("Length closed itemsets: ", len(closedItemSet))

print("\nMaxItemSets are:")
print(maxItemSet)
print("Length max itemsets: ", len(maxItemSet))

print("Time: ", time.time() - start)

ClosedItemSets are: 

[[32, 39], [32, 48], [41, 38, 39], [48, 38, 39], [48, 41, 39]]
Length closed itemsets:  5

MaxItemSets are:
[[32, 39], [32, 48], [41, 38, 39], [48, 38, 39], [48, 41, 39]]
Length max itemsets:  5
Time:  0.015503883361816406


# Redundancy-aware top-k patterns

In [50]:
import numpy as np
import random

def calculate_redundancy(data, centroid, data_point, min_default=50):
    # print(centroid)
    # R(p,q) = S(p) + S(q) − S(p,q)
    new_itemsets = list(data.keys())[centroid] + list(data.keys())[data_point]
    new_itemsets = tuple(set(new_itemsets))
    new_itemsets = tuple(sorted(new_itemsets))
    
    S_p_q = min_default if new_itemsets not in data else data[new_itemsets]
    S_p = data[list(data.keys())[centroid]]
    S_q = data[list(data.keys())[data_point]]

    return S_p + S_q - S_p_q
    # return S_p_q

def get_centroid(data, groupdata, old_centroid):
    min_red = 9999999999999
    min_centroid = old_centroid
    for centroid in groupdata:
        red = 0
        for datapoint in groupdata:
            red = red + calculate_redundancy(data, centroid, datapoint)
        if red < min_red:
            min_centroid = centroid

    return min_centroid

class TopK:
    def __init__(self, tol = 0.0001, max_iter = 300) -> None:
        self.tol = tol
        self.max_iter = max_iter
    
    def fit(self, data, k=2):
        self.centroids = {}
        data_ids = np.arange(len(data))
        data_keys = data.keys()
        cache_ = []

        for i in range(k):
            while True:
                chosen_id = random.randint(0, len(data)-1)
                if chosen_id not in cache_:
                    cache_.append(chosen_id)
                    self.centroids[i] = data_ids[chosen_id]
                    break


        for iter in range(self.max_iter):
            self.groupdata = {}
            for i in range(k):
                self.groupdata[i] = []

            for data_point in data_ids:
                min_dist, min_centroid = 9999999, None
                for j, centroid in self.centroids.items(): 
                    # for each centroid
                    dist = calculate_redundancy(data, centroid, data_point)
                    if dist < min_dist:
                        min_dist = dist
                        min_centroid = j
                index = min_centroid

                self.groupdata[index].append(data_point)
            
            prev_centroids = dict(self.centroids)
            optimized = True

            for index in self.centroids:
                self.centroids[index] = get_centroid(data, self.groupdata[index], self.centroids[index])

            for i in self.centroids:
                original_centroid = prev_centroids[i]
                current_centroid = self.centroids[i]
                if (np.sum((current_centroid - original_centroid)/original_centroid * 1000.0) > self.tol):
                    optimized = False
            
            if optimized:
                break
        
    def predict(self, data):
        data_point = data

        min_dist, min_centroid = 9999999, None
        for j, centroid in self.centroids.items(): 
            # for each centroid
            dist = calculate_redundancy(data, centroid, data_point)
            if dist < min_dist:
                min_dist = dist
                min_centroid = j
        index = min_centroid
        return index, self.centroids[index]



if __name__ == "__main__":
    k = 5
    model = TopK()
    model.fit(dataWithFreq, k=k)
    


In [51]:
for key, data_point in model.centroids.items():
    print(key, "---", list(dataWithFreq.keys())[data_point])

0 --- (38, 39, 48)
1 --- (38,)
2 --- (39, 41)
3 --- (39, 41, 48)
4 --- (41, 48)
